In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

## `SciPy.sparse`: Řídké matice

Scipy.sparse je podmodul knihovny Scipy, který poskytuje datové struktury a algoritmy pro práci s řídkými maticemi. Řídké matice obsahují jen malé procento nenulových prvků, proto se vyplatí s nimi zacházet speciálním způsobem, včetně uložení v paměti. I s tímto nám SciPy může výrazně pomoci.

**Scipy.sparse nabízí několik datových struktur pro reprezentaci řídkých matic, nejčastější jsou tyto:**

1. **coo_matrix**: koordinátní formát matice, kde jsou nenulové prvky uloženy jako souřadnice (řádek, sloupec) a hodnoty.
2. **csr_matrix**: řídká matice ve formátu Compressed Sparse Row, vhodná pro řádkové přístupy (např. násobení).
3. **csc_matrix**: řídká matice ve formátu Compressed Sparse Column, vhodná pro sloupcové přístupy (např. rozklady matice).
4. **lil_matrix**: řídká matice ve formátu List of Lists, vhodná pro rychlé modifikace matice.

Jednotlivé formáty se liší v rychlosti jednotlivých operací a paměťové náročnosti.


In [ ]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix
import scipy.sparse as sparse

Ukážeme si, jak se jednotlivé formáty dají vyrobit a jak jsou uloženy v paměti.

In [ ]:
data = np.array([1, 2, 4, 5, 6])
row_indices = np.array([0, 0, 2, 2, 2])
col_indices = np.array([0, 2, 0, 1, 2])

# což odpovídá plné matici
full_matrix = np.array([[1, 0, 2],
                        [0, 0, 0],
                        [4, 5, 6]])

print("Full matrix:")
print(full_matrix)
print()

# CSR matrix
csr = csr_matrix((data, (row_indices, col_indices)))
print("CSR format:")
print("data:", csr.data)
print("indices:", csr.indices)
print("indptr:", csr.indptr)
print()

# CSC matrix
csc = csc_matrix((data, (row_indices, col_indices)))
print("CSC format:")
print("data:", csc.data)
print("indices:", csc.indices)
print("indptr:", csc.indptr)
print()

# COO matrix
coo = coo_matrix((data, (row_indices, col_indices)))
print("COO format:")
print("data:", coo.data)
print("row:", coo.row)
print("col:", coo.col)
print()

# LIL matrix
lil = lil_matrix((3, 3))
lil[0, 0] = 1
lil[0, 2] = 2
lil[2, 0] = 4
lil[2, 1] = 5
lil[2, 2] = 6
print("LIL format:")
print("data:", lil.data)
print("rows:", lil.rows)
print()


Řídké matice můžeme jednoduše převádět z jednoho formátu do druhého. Navíc můžeme využít funkce `todense()`, která nám řídkou matici převede na obyčejnou NumPy plnou matici.

In [ ]:
A = np.array([[1, 0, 2], [0, 0, 0], [4, 5, 6]])

# Konverze z COO na CSR
A_csr = csr_matrix(A)
print(repr(A_csr))

# Konverze z COO na LIL
A_lil = lil_matrix(A_csr)
print(repr(A_lil))

# Konverze na full matici
A_full = A_csr.todense()
print(repr(A_full))

Řídkou matici je nejlépe vytvořit pomocí seznamu souřadnic nenulových prvků a jejich hodnot, případně přímo pomocí indexů v CSR/CSC formátu. Snažíme se za každou cenu vyhnout se vytváření plných matic.

Při vytváření řídkých matic, kdy chceme hodnoty postupně přidávat, je třeba použít `lil_matrix`. V případě ostatních formátů je přidávání hodnot do existujících matic extrémě pomalé.

Při vytváření řídkých matic pomocí pozic a hodnot nenulových prvků lze mít v seznamu více hodnot se stejnou pozicí. V takovém případě se hodnoty se stejnou pozicí sečtou. Toto je extrémně užitečné při sestavování matic např. pro metodou konečných prvků (potkáte se s ní v předmětu NM2).

In [ ]:
# řídká matice z COO s více hodnotami ve stejných buňkách
indexy_radku = np.array([0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2])
indexy_sloupcu = np.array([0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2])
data = np.array([1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1, 0])

A = coo_matrix((data, (indexy_radku, indexy_sloupcu)))
print(A.todense())

**Vytváření řídkých se specifickými vlastnostmi**
- `sp.eye(n)` - vytvoří jednotkovou matici o velikosti n x n.
- `sp.diags(diagonals, offsets)` - vytvoří matici s hodnotami na diagonálách `diagonals` a offsety `offsets`.

## `scipy.sparse.linalg`: Lineární algebra pro řídké systémy

Scipy.sparse.linalg je podmodul knihovny Scipy, který poskytuje algoritmy pro řešení lineárních systémů s řídkými maticemi. Výhodou řídkých matic je jejich rychlost v případě velkých matic, které jsou ve skutečnosti velmi řídké.

- `spsolve` - řeší soustavu lineárních rovnic s řídkou maticí.
- `spsolve_triangular` - řeší soustavu lineárních rovnic s řídkou maticí, která je trojúhelníková.
- `cg` - sdružené gradienty, iterativní algoritmus pro řešení soustav lineárních rovnic.
- `gmres` - Generalized Minimal Residual, iterativní algoritmus pro řešení soustav lineárních rovnic s řídkými maticemi.
- `minres` - Minimal Residual, iterativní algoritmus pro řešení soustav lineárních rovnic s řídkými maticemi.
- `norm` - vypočítá různé normy matice.
- `eigs` - najde několik největších nebo nejmenších vlastních hodnot a vlastních vektorů řídké matice.
- `svds` - najde několik největších singulárních hodnot a vektorů řídké matice.
- `splu` - rozkládá matici na L a U faktory pomocí LU rozkladu.
- `spilu` - rozkládá matici na L a U faktory pomocí neúplného LU rozkladu (ILU).


In [ ]:
# submoduly je vždy lepší importovat explicitně, pomůžeme tím IDE a zlepšíme čitelnost kódu
import scipy.sparse.linalg as sla

In [ ]:
# vytvoříme třídiagonální řídkou matici s 2 na hlavní diagonále a -1 na vedlejších diagonálách
n = 5
A = sparse.diags([-1, 2, -1], [-1, 0, 1], shape=(n, n), format="csc")
print(repr(A))
print(A.todense())
# a pravou stranu
b = np.ones(n)/n

`spsolve` - přímý sparse řešič (konkrétní řešič se může měnit v závislosti na typu matice a způsobu kompilace SciPy)

In [ ]:
# vyřešíme soustavu přímím řešičem
x = sla.spsolve(A, b)
plt.plot(x)

In [ ]:
# sestavíme LU dekompozici a použijeme ji pro řešení soustavy

# LU dekompozice vrací speciální objekt obsahující všechny potřebné součásti dekompozice
lu_decomp = sla.splu(A)
print(lu_decomp)

In [ ]:

# obtain matrices L, U and permutation P (ordering vector)
L = lu_decomp.L
U = lu_decomp.U
P_permutace = lu_decomp.perm_c

print("Lower triangular matrix L:")
print(repr(L))
print(L.todense())
print()

print("Upper triangular matrix U:")
print(repr(U))
print(U.todense())
print()

print("Permutation matrix P:")
print(P_permutace)

# sestavíme permutační matici P z vektoru P
# syntaxe byla: csc_matrix((data, (indexy_radku, indexy_sloupcu)))
P = csc_matrix((np.ones_like(P_permutace), (P_permutace, np.arange(n))))
print(P.todense())

Kontrola LU rozkladu:

In [ ]:
print("P^T * L * U * P:")
print((P.T @ L @ U @ P).todense())
print()
print("A:")
print(A.todense())


`spsolve_triangular` - řešení soustavy s trojúhelníkovou maticí (výrazně rychlejší než `spsolve`)

In [ ]:
# při řešení s dekompozicí musíme řešit soustavu P^T * L * U * P * x = b
# tedy nejdříve řešíme soustavu L * y = P^T * b
y = sla.spsolve_triangular(L, P @ b, lower=True)

In [ ]:
# a poté soustavu U * P * x = y
x = P.T @ sla.spsolve_triangular(U, y, lower=False)

In [ ]:
plt.plot(x)

Objekt LU rozkladu umožňuje rychle řešit soustavy s danou pravou stranou přímo pomocí metody `solve`.

In [ ]:
# řešení pomocí LU dekompozice lze udělat i jednodušeji
x = lu_decomp.solve(b)
plt.plot(x)

Odhad části spektrálního rozkladu pomocí `eigs` a `eigsh` (rychlejší, pouze pro Hermitovské matice).

In [ ]:
# spočítáme vlastní čísla a vlastní vektory
# eigs počítá vlastní čísla a vlastní vektory pro obecnou matici
# eigsh počítá vlastní čísla a vlastní vektory pro symetrickou (Hermitovskou) matici

w, v = sla.eigsh(A, k=3)
print("Eigenvalues:")
print(w)
print()
print("Eigenvectors:")
print(v)
